**Installing OpenAI CLIP**

In [ ]:
pip install git+https://github.com/openai/CLIP.git

**Import Libraries**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from PIL import Image
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import clip

**Load and Freeze CLIP Model**

In [ ]:
# Initialize CLIP model and the preprocess function for the images

device = "cuda" if torch.cuda.is_available() else "cpu"

clip_model, preprocess = clip.load("RN101", device=device)

# Freeze CLIP model parameters (no training required)

for param in clip_model.parameters():
    param.requires_grad = False

**Loading Affordance Dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Define Paths to Dataset Directories**

In [ ]:
train_images_dir = '/content/drive/My Drive/AffordanceDatasetMain/Train_Set/Images'
train_masks_dir = '/content/drive/My Drive/AffordanceDatasetMain/Train_Set/Masks'
train_annotations_path = '/content/drive/My Drive/AffordanceDatasetMain/Train_Set/annotations.json'

In [ ]:
test_images_dir = '/content/drive/My Drive/AffordanceDatasetMain/Test_Set/Images'
test_annotations_path = '/content/drive/My Drive/AffordanceDatasetMain/Test_Set/annotations.json'

In [ ]:
train_images_dir = '/content/drive/My Drive/Dataset/Train_Set/Images'
train_masks_dir = '/content/drive/My Drive/Dataset/Train_Set/Masks'
train_annotations_path = '/content/drive/My Drive/Dataset/Train_Set/annotations.json'

**Load annotations from the JSON file**

In [ ]:
def load_annotations(annotation_path):
    with open(annotation_path, 'r') as f:
        annotations = json.load(f)
    return annotations

train_annotations = load_annotations(train_annotations_path)
test_annotations = load_annotations(test_annotations_path)

**Dataset Class**

In [ ]:
class AffordanceDataset(Dataset):
    def __init__(self, annotations, images_dir, masks_dir, transform=None):
        self.annotations = annotations
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.transform = transform
        self.preprocess = preprocess

    def load_image(self, image_path):
        image = Image.open(image_path).convert('RGB')
        return image

    def load_mask(self, mask_path):
        mask = Image.open(mask_path).convert('L')
        if self.transform:
            mask = self.transform(mask)
        return mask

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        annotation = self.annotations[idx]
        query = annotation['query']
        object_type = annotation['object_type']
        image_file = annotation['file_name']
        mask_file = annotation['mask_file_name']

        queries = query + " " + object_type

        image_path = os.path.join(self.images_dir, query, object_type, image_file)
        mask_path = os.path.join(self.masks_dir, query, object_type, mask_file)

        image = self.load_image(image_path)
        mask = self.load_mask(mask_path)

        image = self.preprocess(image)

        return image, queries, mask

**Train Dataset Creation**

In [ ]:
transform = transforms.Compose([
    transforms.Resize((60, 60)),
    transforms.ToTensor(),
])

train_dataset = AffordanceDataset(
    annotations=train_annotations,
    images_dir=train_images_dir,
    masks_dir=train_masks_dir,
    transform=transform
)

**Create DataLoader for Batching**

In [ ]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

**Feature Pyramid Network (FPN)**

In [ ]:
class FeaturePyramidNetwork(nn.Module):
    def __init__(self):
        super(FeaturePyramidNetwork, self).__init__()

        # Convolutions for projecting the global visual vector and the feature maps
        self.conv_fs = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1)
        self.conv_f1 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1)
        self.conv_f2 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1)
        self.conv_f3 = nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=3, stride=1, padding=1)

        # Upsample layers (no parameters, bilinear interpolation)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        # Final 1x1 convolution for output
        self.conv_out = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=1, stride=1, padding=1)

        # Batch Normalization and ReLU Activation
        self.bn = nn.BatchNorm2d(num_features=512)
        self.relu = nn.ReLU()

    def forward(self, F_S, F1, F2, F3):

        batch_size = F_S.size(0)

        # Reshape F_S to match the spatial dimensions of F3
        F_S = F_S.view(batch_size, -1, 1, 1)
        F_S = F_S.expand(-1, -1, F3.size(2)//2, F3.size(3)//2)

        # Projected Global Visual Vector F_S
        F_S_proj = self.relu(self.bn(self.conv_fs(F_S)))

        # Projected Feature Maps
        F1_proj = self.relu(self.bn(self.conv_f1(F1)))
        F2_proj = self.relu(self.bn(self.conv_f2(F2)))
        F3_proj = self.relu(self.bn(self.conv_f3(F3)))

        # Implementing Feature Pyramid Network
        F3_fused = F3_proj + self.upsample(F_S_proj)
        F2_fused = F2_proj + self.upsample(F3_fused)
        F1_fused = F1_proj + self.upsample(F2_fused)

        # Final Output Image Features
        F_final = self.conv_out(F1_fused)

        return F_final

**Affordance Head**

In [ ]:
class AffordanceHead(nn.Module):
    def __init__(self):
        super(AffordanceHead, self).__init__()

        # ConvTransposed Layers for affordance map generation
        self.conv_transpose1 = nn.ConvTranspose2d(1, 256, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.conv_transpose2 = nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.conv_out = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=1, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, text_features, image_features):

        # Flatten the Image Features for Matrix Multiplication
        B, C, H, W = image_features.size()
        image_features = image_features.view(B, C, H * W)

        # Compute the Affordance Map (B x H*W)
        affordance_map = torch.bmm(text_features.unsqueeze(1), image_features)

        # Reshape back to B x 1 x H x W
        affordance_map = affordance_map.view(B, 1, H, W)

        # Pass through ConvTransposed Layers
        #affordance_map = self.relu(self.conv_transpose1(affordance_map))
        #affordance_map = self.relu(self.conv_transpose2(affordance_map))

        affordance_map = self.relu(self.conv_out(affordance_map))

        return affordance_map

**AffordanceCLIP Model: Complete Model**

In [ ]:
class AffordanceCLIP(nn.Module):
    def __init__(self, clip_model):
        super(AffordanceCLIP, self).__init__()
        self.clip_model = clip_model
        self.fpn = FeaturePyramidNetwork()
        self.affordance_head = AffordanceHead()

    def forward(self, images, queries):

        # Preprocess images and tokenize queries
        text_inputs = clip.tokenize(queries).to(device)
        images = images.to(torch.float32)

        # Extract Text Features and Image Features
        with torch.no_grad():
            text_features = self.clip_model.encode_text(text_inputs).to(torch.float32)
            image_features = self.clip_model.visual(images).to(torch.float32)


        # Extract Hierarchical Feature Maps
        features = {}

        def get_intermediate_features(name):
            def hook(model, input, output):
                features[name] = output.to(torch.float32)
            return hook

        # Register hooks to specific layers
        self.clip_model.visual.layer1.register_forward_hook(get_intermediate_features('layer1'))
        self.clip_model.visual.layer2.register_forward_hook(get_intermediate_features('layer2'))
        self.clip_model.visual.layer3.register_forward_hook(get_intermediate_features('layer3'))

        # Forward pass to extract features
        with torch.no_grad():
            _ = self.clip_model.visual(images)

        # Access the intermediate layer outputs
        f1 = features['layer1']
        f2 = features['layer2']
        f3 = features['layer3']

        # Pass through the FPN
        refined_features = self.fpn(image_features, f1, f2, f3)

        # Compute affordance map
        affordance_map = self.affordance_head(text_features, refined_features)


        return affordance_map

In [ ]:
# Model Creation
model = AffordanceCLIP(clip_model).to(device)

**Training Loop**

In [ ]:
# Define Binary Cross Entropy
criterion = nn.BCELoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training loop
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0
    correct_pixels = 0
    total_pixels = 0

    for batch_idx, (images, queries, masks) in enumerate(train_loader):
        images = images.to(device)
        masks = masks.to(device)

        print("Min value in Masks:", masks.min())
        print("Max value in Masks:", masks.max())

        # Clear gradients
        optimizer.zero_grad()

        # Forward pass (predict affordance maps)
        affordance_maps = model(images, queries)
        affordance_maps = torch.sigmoid(affordance_maps)

        print("Min value in affordance_maps:", affordance_maps.min())
        print("Max value in affordance_maps:", affordance_maps.max())

        # Compute the loss
        loss = criterion(affordance_maps, masks)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Accumulate the loss over all batches
        epoch_loss += loss.item()

        # Calculate accuracy for the current batch
        with torch.no_grad():
            predicted = (affordance_maps > 0.5).float()  # Binarize predictions
            correct_pixels += (predicted == masks).sum().item()
            total_pixels += masks.numel()  # Total number of pixels

    # Compute the average loss and accuracy for the epoch
    epoch_loss /= len(train_loader)
    accuracy = (correct_pixels / total_pixels) * 100

    # Print the loss and accuracy for the current epoch
    print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Train Accuracy: {accuracy:.2f}%')